In [1]:
import pandas as pd
import numpy as np

from dataprep.eda import plot

# Required for 3A (Distance Travelled)
import geopy.distance
import geocoder

In [2]:
df19 = pd.read_csv('../data/Jan_2019_ontime.csv')
df20 = pd.read_csv('../data/Jan_2020_ontime.csv')

df19['Year'] = 2019
df20['Year'] = 2020

print(df19.shape)
print(df20.shape)

(583985, 23)
(607346, 23)


In [3]:
# find a better way to compare the 2x DFs . 
for i in range(len(df19.columns)):
    if df19.columns[i] == df20.columns[i]:
        print("Match")
    else:
        print("NO MATCH")

Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match


In [4]:
# reset here
df = pd.concat([df19, df20], ignore_index=True, sort=False)

In [5]:
df.shape

(1191331, 23)

df_airport = pd.read_csv('https://github.com/jpatokal/openflights/tree/master/data/airports.dat', \
                     header=None)

df_airport.columns=['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz database time zone','Type','Source']


In [20]:
df_airport = pd.read_csv('../../../packages/openflights/data/airports.dat', header=None)

In [24]:
df_airport.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [25]:
df_airport.columns=['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz database time zone','Type','Source']


In [26]:
df_ap = df_airport[['Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone']]

In [27]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED',
       'DIVERTED', 'DISTANCE', 'Unnamed: 21', 'Year'],
      dtype='object')

In [28]:
df_ap.head(1)

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone
0,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10


In [29]:
df = df.merge(df_ap, how='left', left_on='ORIGIN', right_on='IATA')
df = df.merge(df_ap, how='left', left_on='DEST', right_on='IATA')

In [40]:
df.columns


Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED',
       'DIVERTED', 'DISTANCE', 'Unnamed: 21', 'Year', 'Name_x', 'City_x',
       'Country_x', 'IATA_x', 'ICAO_x', 'Latitude_x', 'Longitude_x',
       'Altitude_x', 'Timezone_x', 'Name_y', 'City_y', 'Country_y', 'IATA_y',
       'ICAO_y', 'Latitude_y', 'Longitude_y', 'Altitude_y', 'Timezone_y'],
      dtype='object')

In [51]:
df[['Latitude_x','Longitude_x']].isna().sum()

Latitude_x     210
Longitude_x    210
dtype: int64

In [55]:
df_dist = (df[['Latitude_x','Longitude_x','Latitude_y','Longitude_y']]
               .dropna()  )

In [57]:
#Iterating through all deliveries to find the total distance travelled (as the crow flies)
distances = range(len(df_dist))
dist_lst = []
cnt = 0
for i in distances:
    #if ((df['ORIGIN'].iloc[i].isna() ==True) | (df['DEST'].iloc[i].isna() ==True)):
    #if (pd.isnull(df['Latitude_x']) != True):
    #    if pd.isnull(df['Longitude_x']) != True :
    origin = (df['Latitude_x'].iloc[i] , df['Longitude_x'].iloc[i])
    dest = (df['Latitude_y'].iloc[i] , df['Longitude_y'].iloc[i])
    dist_lst.append(round(geopy.distance.distance(origin, dest).km,2))
    #else:
    #    cnt += 1
    #    pass

#print(cnt)
df_dist['distance'] = dist_lst  

ValueError: Point coordinates must be finite. (nan, nan, 0.0) has been passed as coordinates.

In [59]:
df.shape

(1191331, 41)

In [62]:
df['ORIGIN'].unique()

array(['GNV', 'MSP', 'DTW', 'TLH', 'ATL', 'DAY', 'JAN', 'LGA', 'JAX',
       'BMI', 'CAK', 'PHL', 'JFK', 'AGS', 'LIT', 'IND', 'EYW', 'CAE',
       'CWA', 'TRI', 'BOS', 'MGM', 'MDT', 'MKE', 'TVC', 'FSM', 'BHM',
       'PIA', 'CVG', 'GTR', 'SDF', 'BNA', 'SAT', 'LFT', 'MSN', 'DSM',
       'EVV', 'ABE', 'DCA', 'BWI', 'ILM', 'TYS', 'LEX', 'CLE', 'ELM',
       'EWN', 'PHF', 'SHV', 'CSG', 'TUL', 'BDL', 'DFW', 'RIC', 'RAP',
       'PIT', 'CRW', 'GPT', 'CHA', 'OMA', 'IAD', 'RDU', 'SAV', 'GRR',
       'BTV', 'LAN', 'OAJ', 'AEX', 'CHO', 'CHS', 'HRL', 'ORF', 'MOT',
       'HSV', 'MCO', 'BTR', 'LAX', 'ORD', 'PHX', 'MIA', 'CMH', 'SFO',
       'SEA', 'STL', 'CLT', 'LAS', 'OAK', 'BUF', 'FLL', 'SJU', 'ALB',
       'PWM', 'SMF', 'MHT', 'TPA', 'EGE', 'OGG', 'HNL', 'MSY', 'IAH',
       'KOA', 'DEN', 'SAN', 'TUS', 'SJC', 'PBI', 'SNA', 'GRI', 'SLC',
       'FAT', 'EWR', 'JAC', 'MCI', 'PNS', 'ELP', 'AUS', 'ABQ', 'PSP',
       'MEM', 'PDX', 'ACT', 'ABI', 'DRT', 'GSO', 'BIL', 'AVP', 'CLL',
       'CID', 'CRP',

In [63]:
df['DEST'].unique()

array(['ATL', 'CVG', 'FSM', 'MSP', 'LGA', 'BMI', 'DAY', 'DTW', 'RIC',
       'EWR', 'AGS', 'OAJ', 'GNV', 'EYW', 'CWA', 'CAE', 'MGM', 'BNA',
       'LNK', 'TVC', 'PIA', 'BHM', 'AVP', 'LAN', 'TYS', 'PIT', 'JAX',
       'GTR', 'BOS', 'MSN', 'JFK', 'DSM', 'EVV', 'MDT', 'OMA', 'AVL',
       'SHV', 'ILM', 'EWN', 'GTF', 'RDU', 'LEX', 'ABE', 'SDF', 'MYR',
       'PWM', 'SYR', 'CSG', 'ELM', 'HSV', 'TUL', 'GRB', 'TLH', 'CRW',
       'TRI', 'LIT', 'GRR', 'IND', 'GPT', 'GSO', 'CHA', 'IAD', 'JAN',
       'ROC', 'TPA', 'CLE', 'AEX', 'BUF', 'CHO', 'BWI', 'BTV', 'HRL',
       'PHF', 'CHS', 'ORF', 'SAV', 'SAT', 'LAX', 'SJC', 'DFW', 'ORD',
       'SEA', 'PHX', 'MIA', 'PHL', 'SFO', 'STL', 'CLT', 'SJU', 'DCA',
       'MCO', 'TUS', 'ALB', 'FLL', 'AUS', 'EGE', 'HNL', 'OGG', 'IAH',
       'LAS', 'KOA', 'MSY', 'SAN', 'PDX', 'BOI', 'ATW', 'SPS', 'SLC',
       'FAT', 'JAC', 'MCI', 'PNS', 'ELP', 'SNA', 'ABQ', 'GSP', 'DEN',
       'RNO', 'MHT', 'ABI', 'DRT', 'CMH', 'BIL', 'CLL', 'ALO', 'SUX',
       'SGF', 'CID',

In [64]:
df['city_pair'] = df['DEST']+df['ORIGIN']

In [65]:
df['city_pair'].head()

0    ATLGNV
1    CVGMSP
2    CVGDTW
3    ATLTLH
4    FSMATL
Name: city_pair, dtype: object

In [70]:
len(df['city_pair'].unique())

5948

In [71]:
df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,Name_y,City_y,Country_y,IATA_y,ICAO_y,Latitude_y,Longitude_y,Altitude_y,Timezone_y,city_pair
0,1,2,9E,20363,9E,N8688C,3280,11953,1195302,GNV,...,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,ATL,KATL,33.636700,-84.428101,1026.0,-5,ATLGNV
1,1,2,9E,20363,9E,N348PQ,3281,13487,1348702,MSP,...,Cincinnati Northern Kentucky International Air...,Cincinnati,United States,CVG,KCVG,39.048801,-84.667801,896.0,-5,CVGMSP
2,1,2,9E,20363,9E,N8896A,3282,11433,1143302,DTW,...,Cincinnati Northern Kentucky International Air...,Cincinnati,United States,CVG,KCVG,39.048801,-84.667801,896.0,-5,CVGDTW
3,1,2,9E,20363,9E,N8886A,3283,15249,1524906,TLH,...,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,ATL,KATL,33.636700,-84.428101,1026.0,-5,ATLTLH
4,1,2,9E,20363,9E,N8974C,3284,10397,1039707,ATL,...,Fort Smith Regional Airport,Fort Smith,United States,FSM,KFSM,35.336601,-94.367401,469.0,-6,FSMATL


In [72]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED',
       'DIVERTED', 'DISTANCE', 'Unnamed: 21', 'Year', 'Name_x', 'City_x',
       'Country_x', 'IATA_x', 'ICAO_x', 'Latitude_x', 'Longitude_x',
       'Altitude_x', 'Timezone_x', 'Name_y', 'City_y', 'Country_y', 'IATA_y',
       'ICAO_y', 'Latitude_y', 'Longitude_y', 'Altitude_y', 'Timezone_y',
       'city_pair'],
      dtype='object')

In [73]:
df.DISTANCE.head()

0    300.0
1    596.0
2    229.0
3    223.0
4    579.0
Name: DISTANCE, dtype: float64

In [75]:
plot(df19)

In [76]:
df19.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21,Year
0,1,2,9E,20363,9E,N8688C,3280,11953,1195302,GNV,...,601.0,0.0,0600-0659,722.0,0.0,0.0,0.0,300.0,NaN,2019
1,1,2,9E,20363,9E,N348PQ,3281,13487,1348702,MSP,...,1359.0,0.0,1400-1459,1633.0,0.0,0.0,0.0,596.0,NaN,2019
2,1,2,9E,20363,9E,N8896A,3282,11433,1143302,DTW,...,1215.0,0.0,1200-1259,1329.0,0.0,0.0,0.0,229.0,NaN,2019
3,1,2,9E,20363,9E,N8886A,3283,15249,1524906,TLH,...,1521.0,0.0,1500-1559,1625.0,0.0,0.0,0.0,223.0,NaN,2019
4,1,2,9E,20363,9E,N8974C,3284,10397,1039707,ATL,...,1847.0,0.0,1900-1959,1940.0,0.0,0.0,0.0,579.0,NaN,2019


Search Results
Local Time
Tuesday
January 1, 2019

In [91]:
df19['date_dt'] = pd.to_datetime(df.Year*10000+100+df.DAY_OF_MONTH,format='%Y%m%d')

In [96]:
df19['date_DoW'] = df19['date_dt'].dt.day_name() #Day of week i.e. Monday, Tuesday

In [99]:
df19.tail()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21,Year,date_dt,date_DoW
583980,31,4,UA,19977,UA,N776UA,200,12016,1201602,GUM,...,0700-0759,1832.0,0.0,0.0,0.0,3801.0,NaN,2019,2019-01-31,Thursday
583981,31,4,UA,19977,UA,N36280,174,12016,1201602,GUM,...,0700-0759,759.0,0.0,0.0,0.0,129.0,NaN,2019,2019-01-31,Thursday
583982,31,4,UA,19977,UA,N36280,117,14955,1495503,SPN,...,0900-0959,933.0,0.0,0.0,0.0,129.0,NaN,2019,2019-01-31,Thursday
583983,31,4,UA,19977,UA,N39726,105,14955,1495503,SPN,...,1800-1859,1854.0,0.0,0.0,0.0,129.0,NaN,2019,2019-01-31,Thursday
583984,31,4,UA,19977,UA,N39726,104,12016,1201602,GUM,...,1600-1659,1719.0,0.0,0.0,0.0,129.0,NaN,2019,2019-01-31,Thursday


In [97]:
plot(df19)

In [102]:
plot(df19[df19['DAY_OF_MONTH'] < 29])

In [ ]:
pd.to_datetime(df.Y*10000+df.M*100+df.D,format='%Y%m%d')

In [90]:
df19['DAY_OF_MONTH'].dtype

dtype('int64')

In [85]:
df19['dt_raw'] = df19['DAY_OF_MONTH']+'/01/'+df19['Year']

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U21'), dtype('<U21')) -> dtype('<U21')

In [84]:
pd.to_datetime(df19.DAY_OF_MONTH+'/01/'+df19.Year)

# DD MM YYYY


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U21'), dtype('<U21')) -> dtype('<U21')

In [42]:
df['Latitude_x'].iloc[595921] == Nan


NameError: name 'Nan' is not defined

In [43]:
df.ORIGIN[df.Latitude_x.isna() == True].unique()

array(['EAR', 'XWA'], dtype=object)

In [44]:
df[df.ORIGIN == 'XWA']

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,Timezone_x,Name_y,City_y,Country_y,IATA_y,ICAO_y,Latitude_y,Longitude_y,Altitude_y,Timezone_y
595921,1,3,OO,20304,OO,N477CA,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
596066,1,3,OO,20304,OO,N8942A,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
597139,1,3,OO,20304,OO,N157SY,5981,16869,1686901,XWA,...,NaN,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672997,5431.0,-7
616207,2,4,OO,20304,OO,N809SK,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
616414,2,4,OO,20304,OO,N685BR,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143234,29,3,OO,20304,OO,N145SY,5817,16869,1686901,XWA,...,NaN,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672997,5431.0,-7
1163677,30,4,OO,20304,OO,N549CA,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
1163933,30,4,OO,20304,OO,N455SW,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
1178249,31,5,OO,20304,OO,N820SK,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6


In [45]:
df_ap[df_ap.IATA == 'JFK']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone
3597,John F Kennedy International Airport,New York,United States,JFK,KJFK,40.639801,-73.7789,13,-5


In [46]:
df_ap[df_ap.IATA == 'EAR']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone


In [47]:
df_ap[df_ap.IATA == 'XWA']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone


In [13]:
df19.columns

'DAY_OF_MONTH'

In [12]:
if df19.columns == df20.columns:
    print("Cols the same")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()